In [ ]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as clr

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import torch.optim as optim

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout

Using TensorFlow backend.


# Import Data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving rnn-challenge-data.npz to rnn-challenge-data (2).npz


In [ ]:
with np.load('rnn-challenge-data.npz') as fh:
    x_train = fh['data_x']
    y_train = fh['data_y']
    x_val = fh['val_x']
    y_val = fh['val_y']
    x_test = fh['test_x']


print(x_train.shape, x_train.dtype)
print(y_train.shape, y_train.dtype)

print(x_val.shape, x_val.dtype)
print(y_val.shape, y_val.dtype)

print(x_test.shape, x_test.dtype)

(400,) <U400
(400,) int64
(100,) <U1200
(100,) int64
(250,) <U2000


# Preprocess Data

In [ ]:
letter_dict = list(dict.fromkeys(x_train[0]))
print(letter_dict)

label_dict = list(dict.fromkeys(y_train))
print(label_dict)


['C', 'T', 'A', 'G']
[2, 0, 4, 3, 1]


In [ ]:
# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(x_train[0]))
# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}
print(char2int)

for key in char2int.keys():
  char2int[key] = char2int[key] - 396
print(char2int)

{'C': 396, 'T': 397, 'A': 398, 'G': 399}
{'C': 0, 'T': 1, 'A': 2, 'G': 3}


In [ ]:
maxlen = len(max(x_train, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 400 characters


Now we can convert our input sequences to sequences of integers instead of characters by mapping them using the dictionaries we created above. This will allow us to one-hot-encode our input sequence subsequently.

In [ ]:
x_train_integer = []
x_val_integer = []
x_test_integer= []

for i in range(len(x_train)):
    # Remove last character for input sequence
    x_train_integer.append(x_train[i])
for i in range(len(x_train)):
    x_train_integer[i] = [char2int[character] for character in x_train_integer[i]]


for i in range(len(x_val)):
    # Remove last character for input sequence
    x_val_integer.append(x_val[i])
for i in range(len(x_val)):
    x_val_integer[i] = [char2int[character] for character in x_val_integer[i]]


for i in range(len(x_test)):
    # Remove last character for input sequence
    x_test_integer.append(x_test[i])
for i in range(len(x_test)):
    x_test_integer[i] = [char2int[character] for character in x_test_integer[i]]

print(x_train[0])
print(x_train_integer[0])
print('---------------------------------')
print(x_train[0])
print(x_train_integer[0])
print('---------------------------------')
print(x_train[0])
print(x_train_integer[0])

CTAGCTGAGCTACTGAGCTACAGTTGACTGACCAGTCAGTGCTAGCTACTGACAGTCTGACAGTTGACCTGACTGATGACCAGTCTAGCAGTGCTACTAGCTAGGCTACAGTCAGTTGACCAGTCTGACAGTCAGTCTGACTGACAGTCAGTCTAGGCTATGACCTGACTGATGACCTGACTGACTGACAGTCTGACTGATGACGCTATGACCTGACTAGCTAGCAGTTGACTGACCTGACAGTGCTACTAGCAGTTGACCAGTGCTACAGTCTGATGACTGACCTGACAGTCTAGGCTACAGTTGACCTGACAGTCAGTGCTACTGACAGTCTAGTGACCAGTCAGTCAGTTGACCTGACTAGCAGTTGACGCTATGACCAGTCTGACAGTGCTACTAG
[0, 1, 2, 3, 0, 1, 3, 2, 3, 0, 1, 2, 0, 1, 3, 2, 3, 0, 1, 2, 0, 2, 3, 1, 1, 3, 2, 0, 1, 3, 2, 0, 0, 2, 3, 1, 0, 2, 3, 1, 3, 0, 1, 2, 3, 0, 1, 2, 0, 1, 3, 2, 0, 2, 3, 1, 0, 1, 3, 2, 0, 2, 3, 1, 1, 3, 2, 0, 0, 1, 3, 2, 0, 1, 3, 2, 1, 3, 2, 0, 0, 2, 3, 1, 0, 1, 2, 3, 0, 2, 3, 1, 3, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 3, 0, 1, 2, 0, 2, 3, 1, 0, 2, 3, 1, 1, 3, 2, 0, 0, 2, 3, 1, 0, 1, 3, 2, 0, 2, 3, 1, 0, 2, 3, 1, 0, 1, 3, 2, 0, 1, 3, 2, 0, 2, 3, 1, 0, 2, 3, 1, 0, 1, 2, 3, 3, 0, 1, 2, 1, 3, 2, 0, 0, 1, 3, 2, 0, 1, 3, 2, 1, 3, 2, 0, 0, 1, 3, 2, 0, 1, 3, 2, 0, 1, 3, 2, 0, 2, 3, 1, 0, 1, 3, 2, 0, 1, 3, 2

Before encoding our input sequence into one-hot vectors, we'll define 3 key variables:

-dict_size: The number of unique characters that we have in our text
This will determine the one-hot vector size as each character will have an assigned index in that vector

-seq_len: The length of the sequences that we're feeding into the model
As we standardised the length of all our sentences to be equal to the longest sentences, this value will be the max length - 1 as we removed the last character input as well

-batch_size: The number of sentences that we defined and are going to feed into the model as a batch

In [ ]:
dict_size = len(letter_dict)
seq_len = maxlen

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)

    print(features.shape)
    print(features.dtype)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i][u][sequence[i][u]] = 1
    return features

In [ ]:
x_train_integer = one_hot_encode(x_train_integer, dict_size, seq_len, len(x_train))
x_val_integer = one_hot_encode(x_val_integer, dict_size, seq_len, len(x_val))
x_test_integer = one_hot_encode(x_test_integer, dict_size, seq_len, len(x_test))

(400, 400, 4)
float32
(100, 400, 4)
float32
(250, 400, 4)
float32


In [ ]:
labels = 5
def y_hot_encode(y):
  y_integer = []
  for i in y:
    l = [0]* labels
    l[i] = 1
    y_integer.append(l)
  return y_integer

In [ ]:
y_train_integer = y_hot_encode(y_train)
y_val_integer = y_hot_encode(y_val)

# RNN Model

In [ ]:
#Hyper parameters
word_vec_length = 400 # Length of the input vector
char_vec_length = 4 # Length of the character vector
output_labels = 5 # Number of output labels

print(f"The input vector will have the shape {word_vec_length}x{char_vec_length}.")
# Out: The input vector will have the shape 23x30.
hidden_nodes = int(2/3 * (word_vec_length * char_vec_length))
print(f"The number of hidden nodes is {hidden_nodes}.")

The input vector will have the shape 400x4.
The number of hidden nodes is 1066.


In [ ]:
# Build the model
print('Build model...')
model = Sequential()
model.add(LSTM(hidden_nodes, return_sequences=False, input_shape=(word_vec_length, char_vec_length)))
model.add(Dropout(0.2))
model.add(Dense(units=output_labels))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

batch_size=100
model.fit(x_train_integer, y_train, batch_size=batch_size, epochs=40, validation_data=(x_val_integer, y_val))

Build model...
Train on 400 samples, validate on 100 samples
Epoch 1/40
400/400 [==============================] - 8s 19ms/step - loss: 1.6207 - acc: 0.1600 - val_loss: 1.6063 - val_acc: 0.2700
Epoch 2/40
400/400 [==============================] - 7s 18ms/step - loss: 1.6054 - acc: 0.2350 - val_loss: 1.6039 - val_acc: 0.2000
Epoch 3/40
400/400 [==============================] - 7s 18ms/step - loss: 1.5990 - acc: 0.2075 - val_loss: 1.5997 - val_acc: 0.3100
Epoch 4/40
400/400 [==============================] - 7s 18ms/step - loss: 1.5955 - acc: 0.2525 - val_loss: 1.5964 - val_acc: 0.2300
Epoch 5/40
400/400 [==============================] - 7s 18ms/step - loss: 1.5866 - acc: 0.2800 - val_loss: 1.5923 - val_acc: 0.2100
Epoch 6/40
400/400 [==============================] - 7s 18ms/step - loss: 1.5762 - acc: 0.2950 - val_loss: 1.5891 - val_acc: 0.2000
Epoch 7/40
400/400 [==============================] - 7s 18ms/step - loss: 1.5692 - acc: 0.2850 - val_loss: 1.5855 - val_acc: 0.1900
Epoch 8/

In [ ]:
loss, accuracy = model.evaluate(x_val_integer, y_val)
print(loss, accuracy)

100/100 [==============================] - 1s 11ms/step
3.396847128868103 0.28999999165534973
